# School District Data Processing and Analysis (Python Version)
This notebook replicates the workflow of the original R scripts, processing and analyzing school district data from multiple sources. The workflow includes:

1. Processing spatial data for school districts

2. Processing EDGE, NCES, CRDC, and educator diversity data

3. Merging and analyzing datasets

4. Preparing data for visualization and further analysis


---

**Note:** You must have all required raw data files (shapefiles, CSVs, Excel files) in the correct folders as referenced in the original R scripts.

In [ ]:
# Import required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from shapely.geometry import shape
import matplotlib.pyplot as plt

## 1. Process Spatial Data for School Districts
This step loads the NCES EDGE school district shapefile, cleans and filters the data, and saves processed outputs for use in later steps.

In [ ]:
# Load and process NCES EDGE school district shapefile
shapefile_path = 'data/raw/EDGE_SCHOOLDISTRICT_TL22_SY2122/EDGE_SCHOOLDISTRICT_TL22_SY2122.shp'
output_csv = 'data/processed/all_school_districts_geo.csv'
output_geojson = 'data/processed/geo/sds_2022.geojson'

# Read shapefile
sds_gdf = gpd.read_file(shapefile_path)

# Filter out undefined school districts
sds_gdf = sds_gdf[sds_gdf['NAME'] != 'School District Not Defined']

# Select relevant columns for CSV output
sd_list = sds_gdf[['STATEFP', 'GEOID', 'NAME', 'INTPTLAT', 'INTPTLON']].copy()
sd_list = sd_list.rename(columns={'STATEFP': 'state', 'GEOID': 'district_id', 'NAME': 'district'})

# Save as CSV
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
sd_list.to_csv(output_csv, index=False)

# Prepare reduced-size GeoDataFrame for GeoJSON output
sds_shape = sds_gdf[['GEOID', 'STATEFP', 'ELSDLEA', 'SCSDLEA', 'UNSDLEA', 'NAME', 'geometry']].copy()
sds_shape = sds_shape.rename(columns={'GEOID': 'district_id', 'STATEFP': 'state', 'NAME': 'district_name'})

# Assign level based on available columns
def get_level(row):
    if pd.notnull(row['ELSDLEA']):
        return 'elementary'
    elif pd.notnull(row['SCSDLEA']):
        return 'secondary'
    elif pd.notnull(row['UNSDLEA']):
        return 'unified'
    else:
        return None
sds_shape['level'] = sds_shape.apply(get_level, axis=1)

# Select and filter columns
sds_shape = sds_shape[['state', 'district_id', 'district_name', 'level', 'geometry']]
exclude_states = ['60', '15', '02', '72', '66', '78', '69']
sds_shape = sds_shape[~sds_shape['state'].isin(exclude_states)]

# Convert CRS to WGS84 (EPSG:4326)
sds_shape = sds_shape.to_crs(epsg=4326)

# Save as GeoJSON
os.makedirs(os.path.dirname(output_geojson), exist_ok=True)
sds_shape.to_file(output_geojson, driver='GeoJSON')

## 2. Process EDGE Socioeconomic and Language Data
This step processes NCES EDGE data on poverty, parental employment, and language spoken at home for each school district. Outputs are merged and saved for later use.

In [ ]:
# Process EDGE Socioeconomic and Language Data
edge_poverty_path = 'data/raw/edge/EDGE_Export_913154920138 - RELEVANT CHILDREN - ENROLLED PUBLIC/CDP03.4_105_USSchoolDistrictAll_913154917544.txt'
edge_employment_path = 'data/raw/edge/EDGE_Export_913155410470 - PARENTS OF RELEVANT CHILDREN - ENROLLED PUBLIC/PDP03.1_205_USSchoolDistrictAll_91315546330.txt'
edge_language_path = 'data/raw/edge/EDGE_Export_913155756929 - RELEVANT CHILDREN - ENROLLED PUBLIC/CDP02.9_105_USSchoolDistrictAll_913155753336.txt'
output_edge_csv = 'data/processed/edge_data_2018_22.csv'

# Load school district list
sds_list = pd.read_csv('data/processed/all_school_districts_geo.csv')[['district_id']]

# Poverty data
raw_poverty = pd.read_csv(edge_poverty_path, delimiter='|')
poverty = raw_poverty[['LEAID', 'Geography', 'CDP03_54pct', 'CDP03_54pctmoe']].rename(columns={
    'LEAID': 'district_id',
    'CDP03_54pct': 'pct_poverty',
    'CDP03_54pctmoe': 'pct_poverty_moe'
})
poverty = poverty.merge(sds_list, on='district_id', how='right')

# Parental employment data
raw_employment = pd.read_csv(edge_employment_path, delimiter='|')
parental_employment = raw_employment[['LEAID', 'PDP03_2est', 'PDP03_2moe', 'PDP03_4est', 'PDP03_4moe', 'PDP03_6est', 'PDP03_6moe', 'PDP03_4pct', 'PDP03_4pctmoe', 'PDP03_6pct', 'PDP03_6pctmoe']].copy()
parental_employment['pct_employed'] = ((parental_employment['PDP03_4est'] + parental_employment['PDP03_6est']) / parental_employment['PDP03_2est']).round(3)
civilian_employment = parental_employment[['LEAID', 'PDP03_4pct', 'PDP03_4pctmoe']].rename(columns={
    'LEAID': 'district_id',
    'PDP03_4pct': 'pct_employed_civilian',
    'PDP03_4pctmoe': 'pct_employed_civilian_moe'
})
civilian_employment = civilian_employment.merge(sds_list, on='district_id', how='right')

# Language data
raw_language = pd.read_csv(edge_language_path, delimiter='|')
language = raw_language.rename(columns={
    'LEAID': 'district_id',
    'CDP02_53est': 'pop',
    'CDP02_53moe': 'pop_moe',
    'CDP02_54pct': 'pct_only_english',
    'CDP02_54pctmoe': 'pct_only_english_moe',
    'CDP02_55pct': 'pct_other_language',
    'CDP02_55pctmoe': 'pct_other_language_moe',
    'CDP02_57pct': 'pct_spanish_athome',
    'CDP02_57pctmoe': 'pct_spanish_athome_moe',
    'CDP02_59pct': 'pct_indo_european_athome',
    'CDP02_59pctmoe': 'pct_indo_european_athome_moe',
    'CDP02_61pct': 'pct_aspi_athome',
    'CDP02_61pctmoe': 'pct_aspi_athome_moe'
})
language_cols = ['district_id', 'pop', 'pop_moe', 'pct_only_english', 'pct_only_english_moe', 'pct_other_language', 'pct_other_language_moe', 'pct_spanish_athome', 'pct_spanish_athome_moe', 'pct_indo_european_athome', 'pct_indo_european_athome_moe', 'pct_aspi_athome', 'pct_aspi_athome_moe']
language = language[language_cols]
for col in ['pct_only_english', 'pct_only_english_moe', 'pct_other_language', 'pct_other_language_moe', 'pct_spanish_athome', 'pct_spanish_athome_moe', 'pct_indo_european_athome', 'pct_indo_european_athome_moe', 'pct_aspi_athome', 'pct_aspi_athome_moe']:
    language[col] = language[col] / 100
language = language.merge(sds_list, on='district_id', how='right')

# Merge all EDGE data
edge_data = poverty.merge(civilian_employment, on='district_id', how='left').merge(language, on='district_id', how='left')

# Save processed EDGE data
os.makedirs(os.path.dirname(output_edge_csv), exist_ok=True)
edge_data.to_csv(output_edge_csv, index=False)